# Отток клиентов банка

## Подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.utils import shuffle

In [2]:
try:
    data = pd.read_csv('/datasets/Churn.csv')
except: 
    data = pd.read_csv('C:/documents/datasets/Churn.csv')

In [3]:
display(data)
display(data.info())
display(data.describe())

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


None

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [4]:
data.columns = (data.columns.str.replace('(?<=[a-z])(?=[A-Z])', '_', regex=True).str.lower())
data

,row_number,customer_id,surname,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


In [5]:
data['gender'].value_counts(normalize=True)

Male      0.5457
Female    0.4543
Name: gender, dtype: float64

In [6]:
data['geography'].value_counts(normalize=True)

France     0.5014
Germany    0.2509
Spain      0.2477
Name: geography, dtype: float64

In [7]:
data.duplicated().value_counts()

False    10000
dtype: int64

In [8]:
data['customer_id'].duplicated().value_counts()

False    10000
Name: customer_id, dtype: int64

In [9]:
data.loc[:, ['surname', 'geography', 'age', 'gender', 'estimated_salary']].duplicated().value_counts()

False    10000
dtype: int64

In [10]:
data['tenure'] = data['tenure'].fillna(0)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   row_number        10000 non-null  int64  
 1   customer_id       10000 non-null  int64  
 2   surname           10000 non-null  object 
 3   credit_score      10000 non-null  int64  
 4   geography         10000 non-null  object 
 5   gender            10000 non-null  object 
 6   age               10000 non-null  int64  
 7   tenure            10000 non-null  float64
 8   balance           10000 non-null  float64
 9   num_of_products   10000 non-null  int64  
 10  has_cr_card       10000 non-null  int64  
 11  is_active_member  10000 non-null  int64  
 12  estimated_salary  10000 non-null  float64
 13  exited            10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [11]:
data_ohe = data.drop('surname', axis=1)
data_ohe = data_ohe.drop('customer_id', axis=1)
data_ohe = data_ohe.drop('row_number', axis=1)

In [12]:
target = data_ohe['exited']
features = data_ohe.drop('exited', axis=1)
features_train, features_valid, target_train, target_valid = train_test_split(features, target, 
                                                                              test_size=0.4, random_state=12345)
features_test, features_valid, target_test, target_valid = train_test_split(features_valid, target_valid, 
                                                                              test_size=0.5, random_state=12345)
features_train = pd.get_dummies(features_train, drop_first=True)
features_valid = pd.get_dummies(features_valid, drop_first=True)
features_test = pd.get_dummies(features_test, drop_first=True)


print(features_train.shape)
print(features_valid.shape)
print(features_test.shape)
print(target_train.shape)
print(target_valid.shape)
print(target_test.shape)

(6000, 11)
(2000, 11)
(2000, 11)
(6000,)
(2000,)
(2000,)


Загружен датасет с 10 тыс записями клиентов банка. В нем представлены клиенты со следующими параметрами:
1. Крединтный рейтинг от 350 до 850. В среднем 650
2. Возраст от 18 до 92 лет. В среднем 37-39
3. Клиенты пользуются услугами банка от 0 до 10 лет. В среднем 5 лет
4. Баланс на счете от 0 до 250 тыс. В среднем 76 тыс
5. Кол-во продуктов, которыми пользуются от 1 до 4. В среднем 1
6. У большинства клиентов (около 70%) есть кредитные карты
7. Более половины клиентов (окло 51%) являются активными пользователями
8. Зарплата от 11 до 199992, в среднем 100000.
9. Около 20% клиентов банка из датасета ушли из банка.
10. 55% клиентов мужчины и 45% женщины
11. 50% клиентов из Франции, 25% из Германнии и 25% из Испании


Явных дубликатов не обнаружено. Неявных дубликактов так же нет, как по Id клиента, так и по столбца фамилия, страна, возвраст, пол и зарплата. Перевели названия столбцов к нижнему регистру

Есть столбец с пустыми данными по кол-ву лет использования услуг. Возможно при выгрузке данных возинка тезническая ошибка или человеческий фактор при внесении данных в систему. Заменим данные на 0. Так же уберем фаилии клиентов(тк есть точное соотве-ие фамилии и id и проводим прямое кодирование по двум оставшимся столбцам оbject: Страна и пол, для дальнейшего обучения. Так же разделим выборку на обучающую, валидационну и тестовую в пропорции 60 на 20 на 20.

## Исследование задачи

In [13]:
target_train.value_counts(normalize=True)

0    0.800667
1    0.199333
Name: exited, dtype: float64

In [14]:
model = LogisticRegression(random_state=12345)
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
print('Значнеие f1 меры:',f1_score(target_valid, predictions_valid))

Значнеие f1 меры: 0.13737373737373737


In [15]:
model = DecisionTreeClassifier(random_state=12345)
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
f1_score(target_valid, predictions_valid)

0.47515151515151516

In [16]:
f1 = 0
dep =1
tr=10
for depth in range(1, 16, 1):
    for i in range(10, 100, 10):
        model = RandomForestClassifier(n_estimators=i, max_depth=depth, random_state=12345)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        a = f1_score(target_valid, predictions_valid)
        if f1<a:
            dep = depth
            tr = i
            f1 = a
print('Лучашая глубина:',dep)
print('Лучшее кол-во деревьев:',tr)
print('Лучшая f1 мера:',f1)

Лучашая глубина: 15
Лучшее кол-во деревьев: 30
Лучшая f1 мера: 0.5446293494704993


В таргете у нас виден дисбаланс классво 20 на 80. Мы обучили 3 модели на обучаующей выборке и обнаружили, что лучший результат f1 меры получается в модели случайный лес с 30 деревьями и глубиной 15. Полученная f1 мера 0.54 меньше метрики 0.59, заявленной в ТЗ.

## Борьба с дисбалансом

In [17]:
model = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
print('Значнеие f1 меры:',f1_score(target_valid, predictions_valid))

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('Значнеие AUC-ROC меры:', auc_roc)

Значнеие f1 меры: 0.4293695131683959
Значнеие AUC-ROC меры: 0.6849450808084896


In [18]:
model = DecisionTreeClassifier(random_state=12345)
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
print('Значнеие f1 меры:',f1_score(target_valid, predictions_valid))

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('Значнеие AUC-ROC меры:', auc_roc)

Значнеие f1 меры: 0.47515151515151516
Значнеие AUC-ROC меры: 0.6663645998701787


In [19]:
f1 = 0
dep =1
tr=10
for depth in range(1, 16, 1):
    for i in range(10, 100, 10):
        model = RandomForestClassifier(n_estimators=i, max_depth=depth, random_state=12345, class_weight='balanced')
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        a = f1_score(target_valid, predictions_valid)
        if f1<a:
            dep = depth
            tr = i
            f1 = a
            
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('Значнеие AUC-ROC меры:', auc_roc)
print('Лучашая глубина:',dep)
print('Лучшее кол-во деревьев:',tr)
print('Лучшая f1 мера:',f1)

Значнеие AUC-ROC меры: 0.8503487634749525
Лучашая глубина: 9
Лучшее кол-во деревьев: 60
Лучшая f1 мера: 0.6157303370786517


Введя дополнительный гиперпараметр веса класса мы получили лучшую f1 меру 0.62 и AUC-ROC 0.85 в модели случайного леса с 60 деревьями и глубиной 9, соответвующую ТЗ (больше 0.59)

In [20]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

In [21]:
model = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model.fit(features_upsampled, target_upsampled)
predictions_valid = model.predict(features_valid)
print('Значнеие f1 меры:',f1_score(target_valid, predictions_valid))
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('Значнеие AUC-ROC меры:', auc_roc)

Значнеие f1 меры: 0.4293695131683959
Значнеие AUC-ROC меры: 0.6849450808084896


In [22]:
model = DecisionTreeClassifier(random_state=12345)
model.fit(features_upsampled, target_upsampled)
predictions_valid = model.predict(features_valid)
print('Значнеие f1 меры:',f1_score(target_valid, predictions_valid))

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('Значнеие AUC-ROC меры:', auc_roc)

Значнеие f1 меры: 0.49388753056234713
Значнеие AUC-ROC меры: 0.6775785486102679


In [23]:
f1 = 0
dep =1
tr=10
for depth in range(1, 16, 1):
    for i in range(10, 100, 10):
        model = RandomForestClassifier(n_estimators=i, max_depth=depth, random_state=12345, class_weight='balanced')
        model.fit(features_upsampled, target_upsampled)
        predictions_valid = model.predict(features_valid)
        a = f1_score(target_valid, predictions_valid)
        if f1<a:
            dep = depth
            tr = i
            f1 = a
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('Значнеие AUC-ROC меры:', auc_roc)
print('Лучашая глубина:',dep)
print('Лучшее кол-во деревьев:',tr)
print('Лучшая f1 мера:',f1)

Значнеие AUC-ROC меры: 0.8518178724603528
Лучашая глубина: 11
Лучшее кол-во деревьев: 30
Лучшая f1 мера: 0.6252771618625277


Мы произвели увеличение выборки для попытки ликвидировать дисбаланс классов. В сравнении с предыдущим методом, это не привело к существенному улучшению метрик. Лучшая модель - случайный лес с 30 деревьями и глубиной 11 дала метрику f1 0.63 и AUC-ROC 0.85

## Тестирование модели

In [24]:
model = RandomForestClassifier(n_estimators=60, max_depth=9, random_state=12345, class_weight='balanced')
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)

print('Значнеие f1 меры:',f1_score(target_test, predictions_test))

probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
auc_roc = roc_auc_score(target_test, probabilities_one_test)
print('Значнеие AUC-ROC меры:', auc_roc)

Значнеие f1 меры: 0.6208476517754868
Значнеие AUC-ROC меры: 0.8541667926856562


При использовании улчшей модели случайный лес с 60 деревьями и глубиной 9, с гиперпарамтров вес классов мы получили на тестовой выборке меру f1=0.62 и AUC-ROC=0.85. Достигнуты показатели заданные в ТЗ.